In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install tweet-preprocessor
!pip install transformers
!pip install sentence_transformers
!pip install ranky
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=e8f1e76c8d7254770606c1b166137fbf9c3fda8c053d98e9399c024528466514
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e746291

## Imports

In [3]:
import preprocessor as p
import re
import nltk
import numpy as np
import pandas as pd
import transformers
import torch
import string
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
# from py_rankaggregation import CombANZ
import ranky as rk
from keybert import KeyBERT
import spacy
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Preprocessing

In [6]:
# download the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
# Preprocessing
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Removing URLs, retweets, mentions, stopwords
    rtwords = ['rt', 'RT', 'retweet', 'Retweet', 'RETWEET']
    stops = stopwords.words('english') + rtwords
    
    mess = re.sub(r"http\S+", "", mess)
    mess = re.sub("@[A-Za-z0-9_]+","", mess)
    mess = re.sub("#","", mess)

    # Define a regular expression pattern that matches English words
    pattern = re.compile(r'[a-zA-Z]+')
    
    # Find all matches of the pattern in the text
    matches = re.findall(pattern, mess)
    
    # Join the matches back into a single string
    english_text = ' '.join(matches)
    
    # # STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # # Check characters to see if they are in punctuation
    nopunc = [char for char in english_text if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # # Now just remove any stopwords
    # return ' '.join([word for word in nopunc.split() if word.lower() not in stops])
    
    # define the tokenizer, stemmer, and lemmatizer
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()

    # convert to lowercase
    tweet = nopunc.lower()
    # tokenize the tweet
    # tokens = word_tokenize(tweet)
    # remove stop words
    # tokens = [token for token in tokens if token not in stop_words]
    # # stem the tokens
    # tokens = [stemmer.stem(token) for token in tokens]
    # # lemmatize the tokens
    # tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # print("here\n")
    doc = nlp(tweet)
    # doc = [token for token in doc if detect(token) == 'en']
    doc = [token for token in doc if token not in stop_words]
    
    # print("here2\n")
    # Lemmatize the tokens
    lemmas = [token.lemma_ for token in doc]
    
    # print("here3\n")
    # Join the lemmas back into a single string
    processed_tweet = ' '.join(lemmas)

    # print("here4\n")
    # # re-join the tokens into a string
    # processed_tweet = ' '.join(tokens)
    return processed_tweet

## Loading the dataset

In [33]:
cyber_tweets = pd.read_csv('/content/drive/MyDrive/SMAI-project/data/cybercrime_tweets.csv')
cyber_tweets = cyber_tweets.iloc[:10000]

In [34]:
cyber_tweets

,Tweet,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1
1,is upset that he can't update his Facebook by ...,1
2,@Kenichan I dived many times for the ball. Man...,1
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",1
...,...,...
9995,Aww that's sad,1
9996,stupid dvds stuffing up the good bits in jaws.,1
9997,@Dandy_Sephy No. Only close friends and family...,1
9998,CRAP! After looking when I last tweeted... WHY...,1


In [35]:
cyber_tweets = cyber_tweets[['Tweet']]
cyber_tweets

,Tweet
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,is upset that he can't update his Facebook by ...
2,@Kenichan I dived many times for the ball. Man...
3,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all...."
...,...
9995,Aww that's sad
9996,stupid dvds stuffing up the good bits in jaws.
9997,@Dandy_Sephy No. Only close friends and family...
9998,CRAP! After looking when I last tweeted... WHY...


In [36]:
raw_tweets = cyber_tweets['Tweet'].tolist()
raw_tweets

["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!",
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds',
 'my whole body feels itchy and like its on fire ',
 "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. ",
 '@Kwesidei not the whole crew ',
 'Need a hug ',
 "@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?",
 "@Tatiana_K nope they didn't have it ",
 '@twittera que me muera ? ',
 "spring break in plain city... it's snowing ",
 'I just re-pierced my ears ',
 "@caregiving I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . .",
 '@octolinz16 It it counts, idk why I did either. you never talk to me anymore ',
 "@smarrison i wo

In [37]:
cyber_tweets['Tweet'] = cyber_tweets['Tweet'].apply(text_process)

<ipython-input-37-3e436a632c6c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyber_tweets['Tweet'] = cyber_tweets['Tweet'].apply(text_process)


In [38]:
cyber_tweets

,Tweet
0,awww that s a bummer you shoulda get david car...
1,be upset that he can t update his facebook by ...
2,I dive many time for the ball manage to save t...
3,my whole body feel itchy and like its on fire
4,no it s not behave at all I m mad why be I her...
...,...
9995,aww that s sad
9996,stupid dvds stuff up the good bit in jaw
9997,no only close friend and family I m afraid I m...
9998,crap after look when I last tweet why be I up ...


In [39]:
tweets = cyber_tweets['Tweet'].tolist()
tweets

['awww that s a bummer you shoulda get david carr of third day to do it d',
 'be upset that he can t update his facebook by texte it and might cry as a result school today also blah',
 'I dive many time for the ball manage to save the rest go out of bound',
 'my whole body feel itchy and like its on fire',
 'no it s not behave at all I m mad why be I here because I can t see you all over there',
 'not the whole crew',
 'need a hug',
 'hey long time no see yes rain a bit only a bit lol I m fine thank how s you',
 'nope they didn t have it',
 'que I muera',
 'spring break in plain city it s snow',
 'I just re pierce my ear',
 'I couldn t bear to watch it and I think the ua loss be embarrass',
 'it it count idk why I do either you never talk to I anymore',
 'I would ve be the first but I didn t have a gun not really though zac snyder s just a doucheclown',
 'I wish I get to watch it with you I miss you and how be the premiere',
 'hollis death scene will hurt I severely to watch on film wr

## Get tri-grams from each tweet

In [40]:
def get_ngrams(sentence, n):
    words = sentence.split()
    phrases = []
    for i in range(len(words)-(n-1)):
        phrase = " ".join(words[i:i+n])
        phrases.append(phrase)
    return phrases

In [41]:
# get n-grams for each tweet
ngrams = []
for tweet in tweets:
    ngrams.append(get_ngrams(tweet, 3))

# flatten list of lists
ngrams = [item for sublist in ngrams for item in sublist]
print(len(ngrams))

# get unique n-grams
unique_ngrams = list(set(ngrams))
print(len(unique_ngrams))

116814
93922


## Sentence Transformer Models

In [42]:
# Initializing the sentence transformer models
xlm_dist = SentenceTransformer('sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1')

distilbert = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')

para_minilm = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

## Getting the embeddings of tweets and tri-grams

In [43]:
xlm_embeddings = xlm_dist.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of tweets:")
print(xlm_embeddings)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]


XLM-Roberta embeddings of tweets:
tensor([[ 0.1495,  0.2681,  0.1877,  ...,  0.2291, -0.0883, -0.2288],
        [ 0.0376, -0.0510,  0.0050,  ..., -0.0747, -0.2404,  0.4924],
        [-0.1438,  0.1809,  0.0552,  ...,  0.1718,  0.0531, -0.3988],
        ...,
        [-0.2828,  0.2506, -0.0438,  ..., -0.0337, -0.1428, -0.0493],
        [-0.0521, -0.2226,  0.0645,  ..., -0.3075, -0.0821, -0.1342],
        [ 0.1253,  0.0274,  0.0018,  ...,  0.1425, -0.2470,  0.0678]],
       device='cuda:0')


In [44]:
distilbert_embeddings = distilbert.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of tweets:")
print(distilbert_embeddings)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]


Distilbert embeddings of tweets:
tensor([[ 0.2322, -0.0725,  0.1727,  ..., -0.1313,  0.1304,  0.2054],
        [ 0.5311,  0.2185,  0.0199,  ..., -0.0939, -0.0306,  0.1499],
        [-0.0214, -0.1192,  0.0185,  ...,  0.0464, -0.0052,  0.0239],
        ...,
        [ 0.2354,  0.2511,  0.2202,  ..., -0.0154, -0.1651, -0.1091],
        [ 0.3951,  0.1351,  0.1264,  ...,  0.1650,  0.1020, -0.1212],
        [ 0.3948, -0.3449,  0.3356,  ...,  0.0778, -0.0676, -0.1789]],
       device='cuda:0')


In [45]:
minilm_embeddings = para_minilm.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of tweets:")
print(minilm_embeddings)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]


Para MiniLM embeddings of tweets:
tensor([[ 0.1539, -0.1019, -0.3250,  ..., -0.3669, -0.1193, -0.0512],
        [-0.0928, -0.1183,  0.0422,  ..., -0.2036,  0.3071,  0.2680],
        [ 0.6898,  0.4873,  0.1306,  ..., -0.1769,  0.1186, -0.2207],
        ...,
        [-0.1609,  0.2741,  0.1600,  ..., -0.3430,  0.3345, -0.0372],
        [-0.6468, -0.1845,  0.2739,  ...,  0.0360,  0.0292,  0.3058],
        [ 0.0403, -0.3755,  0.4809,  ..., -0.0781, -0.8130,  0.4359]],
       device='cuda:0')


In [46]:
xlm_ngram_embeddings = xlm_dist.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of n-grams:")
print(xlm_ngram_embeddings)

Batches:   0%|          | 0/2936 [00:00<?, ?it/s]


XLM-Roberta embeddings of n-grams:
tensor([[-0.0876, -0.0041, -0.0306,  ..., -0.0296, -0.3056, -0.0128],
        [-0.1558, -0.2221, -0.0202,  ..., -0.0665, -0.0769,  0.0601],
        [-0.0087,  0.0074,  0.3535,  ...,  0.0412,  0.2615,  0.1361],
        ...,
        [-0.0054,  0.0505,  0.0364,  ...,  0.0630, -0.1749, -0.5041],
        [ 0.0509,  0.2595,  0.2151,  ...,  0.1744,  0.0102,  0.1997],
        [-0.1478,  0.1063,  0.2417,  ..., -0.0047,  0.0972, -0.0792]],
       device='cuda:0')


In [47]:
distilbert_ngram_embeddings = distilbert.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of n-grams:")
print(distilbert_ngram_embeddings)

Batches:   0%|          | 0/2936 [00:00<?, ?it/s]


Distilbert embeddings of n-grams:
tensor([[ 0.6970, -0.0012,  0.2851,  ...,  0.2611, -0.4762, -0.0490],
        [ 0.3158, -0.0638, -0.0348,  ...,  0.2839,  0.0984, -0.7988],
        [ 0.3622, -0.0384,  0.2268,  ...,  0.2660,  0.0078, -0.0958],
        ...,
        [ 0.1249, -0.0549, -0.0581,  ..., -0.0348, -0.2899,  0.1354],
        [-0.1346, -0.0224, -0.0524,  ...,  0.3637, -0.1624, -0.5112],
        [-0.2476,  0.1593, -0.3802,  ..., -0.0371, -0.5393, -0.2880]],
       device='cuda:0')


In [48]:
minilm_ngram_embeddings = para_minilm.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of n-grams:")
print(minilm_ngram_embeddings)

Batches:   0%|          | 0/2936 [00:00<?, ?it/s]


Para MiniLM embeddings of n-grams:
tensor([[-0.0767,  0.2798,  0.0146,  ..., -0.2569,  0.5965,  0.5502],
        [ 0.2074,  1.0394,  0.1745,  ..., -0.2050,  0.2718, -0.0882],
        [ 0.7149,  0.5088,  0.1890,  ...,  0.0057, -0.4144, -0.4326],
        ...,
        [ 0.0706,  0.2039,  0.3208,  ..., -0.0831,  0.5483, -0.1034],
        [ 0.5993,  0.4527,  0.7812,  ...,  0.1395, -0.1252, -0.2301],
        [-0.6201, -0.6051,  0.1117,  ...,  0.1730, -0.4244, -0.1850]],
       device='cuda:0')


In [49]:
torch.save(xlm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/cybercrime_data_embeddings/xlm_tweet_embeddings.pt")
torch.save(xlm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/cybercrime_data_embeddings/xlm_ngram_embeddings.pt")
torch.save(distilbert_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/cybercrime_data_embeddings/distilbert_tweet_embeddings.pt")
torch.save(distilbert_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/cybercrime_data_embeddings/distilbert_ngram_embeddings.pt")
torch.save(minilm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/cybercrime_data_embeddings/minilm_tweet_embeddings.pt")
torch.save(minilm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/cybercrime_data_embeddings/minilm_ngram_embeddings.pt")

## Computing average Cosine Similarity between each tri-gram and the document

In [50]:
xlm_similarities = []
for ngram in xlm_ngram_embeddings:
    xlm_similarities.append(util.pytorch_cos_sim(xlm_embeddings, ngram).mean().item())

In [51]:
xlm_similarities

[0.15859030187129974,
 0.14049369096755981,
 0.1692548394203186,
 0.12013709545135498,
 0.1662493795156479,
 0.16225726902484894,
 0.21725180745124817,
 0.1492088884115219,
 0.12805642187595367,
 0.0998007133603096,
 0.20981022715568542,
 0.1501641571521759,
 0.10689860582351685,
 0.21599081158638,
 0.1634836196899414,
 0.22211532294750214,
 0.13817183673381805,
 0.1666218489408493,
 0.21284110844135284,
 0.16787390410900116,
 0.14052420854568481,
 0.18890433013439178,
 0.14944328367710114,
 0.17001141607761383,
 0.16735659539699554,
 0.1527031511068344,
 0.12877370417118073,
 0.15835900604724884,
 0.1010618656873703,
 0.17475102841854095,
 0.12603901326656342,
 0.19133161008358002,
 0.16239319741725922,
 0.09908312559127808,
 0.19339759647846222,
 0.0917603150010109,
 0.18054607510566711,
 0.06833943724632263,
 0.13856056332588196,
 0.15512804687023163,
 0.1886821836233139,
 0.22180575132369995,
 0.0314970500767231,
 0.13692867755889893,
 0.05665687099099159,
 0.2107633799314499,
 0.1

In [52]:
# minilm_similarity_matrix = util.pytorch_cos_sim(minilm_embeddings, minilm_ngram_embeddings)
# minilm_similarity_matrix

minilm_similarities = []
for ngram in minilm_ngram_embeddings:
    minilm_similarities.append(util.pytorch_cos_sim(minilm_embeddings, ngram).mean().item())

In [53]:
minilm_similarities

[0.17118476331233978,
 0.13256718218326569,
 0.1370398998260498,
 0.11329255253076553,
 0.2379395216703415,
 0.1611931174993515,
 0.1860484629869461,
 0.08080048859119415,
 0.13476264476776123,
 0.008197791874408722,
 0.2082599550485611,
 0.149726003408432,
 0.053206101059913635,
 0.14543898403644562,
 0.17844952642917633,
 0.1380286067724228,
 0.15385907888412476,
 0.20219536125659943,
 0.1723308265209198,
 0.18786536157131195,
 0.08384370803833008,
 0.18976236879825592,
 0.13202330470085144,
 0.135050967335701,
 0.13192953169345856,
 0.1189781203866005,
 0.13333408534526825,
 0.14186818897724152,
 0.08339230716228485,
 0.1047465056180954,
 0.18181373178958893,
 0.18399518728256226,
 0.10932809859514236,
 0.056066930294036865,
 0.18803372979164124,
 0.10474643111228943,
 0.11347946524620056,
 0.09092649072408676,
 0.10833948850631714,
 0.09037221223115921,
 0.17376139760017395,
 0.13825342059135437,
 0.06557715684175491,
 0.11072178930044174,
 0.04278627038002014,
 0.16872218251228333

In [54]:
# minilm_similarity_matrix.shape

In [55]:
# # distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# from sklearn.metrics.pairwise import cosine_similarity
# distilbert_embeddings = distilbert_embeddings.to("cpu")
# distilbert_ngram_embeddings = distilbert_ngram_embeddings.to("cpu")
# distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# distilbert_similarity_matrix

distilbert_similarities = []
for ngram in distilbert_ngram_embeddings:
    distilbert_similarities.append(util.pytorch_cos_sim(distilbert_embeddings, ngram).mean().item())

In [56]:
# distilbert_similarity_matrix.shape

In [57]:
distilbert_similarities

[0.6550306677818298,
 0.6470041871070862,
 0.6637439131736755,
 0.6631298661231995,
 0.7087643146514893,
 0.6694718599319458,
 0.6886216998100281,
 0.6630187034606934,
 0.6353247165679932,
 0.6396475434303284,
 0.6718652248382568,
 0.6801270246505737,
 0.6305164694786072,
 0.6801376938819885,
 0.665433943271637,
 0.6706095933914185,
 0.6829324960708618,
 0.6822145581245422,
 0.6646990180015564,
 0.6499386429786682,
 0.6658792495727539,
 0.6416685581207275,
 0.6681079864501953,
 0.6833062171936035,
 0.6767284274101257,
 0.6645371317863464,
 0.6356470584869385,
 0.6507652401924133,
 0.6682655215263367,
 0.6871538758277893,
 0.6768055558204651,
 0.6472009420394897,
 0.6797121167182922,
 0.6345715522766113,
 0.7172277569770813,
 0.6659916043281555,
 0.6823720932006836,
 0.6469895839691162,
 0.6363211870193481,
 0.6655446887016296,
 0.689102053642273,
 0.6751915216445923,
 0.6227197051048279,
 0.6581523418426514,
 0.6300505995750427,
 0.6514040231704712,
 0.6692236065864563,
 0.620126724243

In [58]:
xlm_similarities = np.array(xlm_similarities)
distilbert_similarities = np.array(distilbert_similarities)
minilm_similarities = np.array(minilm_similarities)

## Normailizing the similarities using MinMaxScaler

In [59]:
scaler = MinMaxScaler()
xlm_similarities = scaler.fit_transform(xlm_similarities.reshape(-1, 1))
distilbert_similarities = scaler.fit_transform(distilbert_similarities.reshape(-1, 1))
minilm_similarities = scaler.fit_transform(minilm_similarities.reshape(-1, 1))

## Putting keyphrases and scores in a dataframe and ranking them individually for each model

In [60]:
xlm_scores_df = pd.DataFrame(list(zip(unique_ngrams, xlm_similarities)), columns=['keyphrase', 'score'])
distilbert_scores_df = pd.DataFrame(list(zip(unique_ngrams, distilbert_similarities)), columns=['keyphrase', 'score'])
minilm_scores_df = pd.DataFrame(list(zip(unique_ngrams, minilm_similarities)), columns=['keyphrase', 'score'])

In [61]:
xlm_scores_df

,keyphrase,score
0,just give an,[0.5158533654500075]
1,to hell amp,[0.46047498498120093]
2,night and easter,[0.5484884716063424]
3,episode definitely one,[0.39818070802832595]
4,now but my,[0.5392913073986398]
...,...,...
93917,this then evening,[0.5487631649134339]
93918,in a red,[0.27286611381123627]
93919,she must have,[0.3424285686465753]
93920,sad we didn,[0.5550276679891688]


## Removing keyphrases with scores less than threshold

In [62]:
remove_list = []
threshold = 0.3

anomalies1 = xlm_scores_df[xlm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies1)

anomalies2 = distilbert_scores_df[distilbert_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies2)

anomalies3 = minilm_scores_df[minilm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies3)

remove_list = [item for sublist in remove_list for item in sublist]
remove_list = list(set(remove_list))

len(remove_list)

13144

In [63]:
# removing the keyphrases from remove_list from all 3 dataframes
xlm_scores_df = xlm_scores_df[~xlm_scores_df['keyphrase'].isin(remove_list)]
distilbert_scores_df = distilbert_scores_df[~distilbert_scores_df['keyphrase'].isin(remove_list)]
minilm_scores_df = minilm_scores_df[~minilm_scores_df['keyphrase'].isin(remove_list)]

In [64]:
xlm_rank_df = xlm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
xlm_rank_df['rank'] = xlm_rank_df.index + 1
xlm_rank_df

,keyphrase,score,rank
0,sad I wolud,[1.0],1
1,vomitte on my,[0.9543678055545304],2
2,sy pn mau,[0.9492227780247942],3
3,I m afriad,[0.9386047690701355],4
4,manu ginobili be,[0.9312539871163241],5
...,...,...,...
80773,hot cross bun,[0.3000283525421951],80774
80774,reach get a,[0.30002744054582364],80775
80775,fish and the,[0.3000131906025192],80776
80776,ontd the entry,[0.3000130766029727],80777


In [65]:
distilbert_rank_df = distilbert_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
distilbert_rank_df['rank'] = distilbert_rank_df.index + 1
distilbert_rank_df

,keyphrase,score,rank
0,that but my,[1.0],1
1,that but I,[0.9937071822648473],2
2,on that but,[0.9910811345672585],3
3,this but I,[0.9837366170516852],4
4,that too and,[0.9832844074911273],5
...,...,...,...
80773,just be pure,[0.30025800034335015],80774
80774,thin end of,[0.3002555609121398],80775
80775,coat to cotton,[0.30008175143842575],80776
80776,a speed ticket,[0.30004180575236195],80777


In [66]:
minilm_rank_df = minilm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
minilm_rank_df['rank'] = minilm_rank_df.index + 1
minilm_rank_df

,keyphrase,score,rank
0,lame friday I,[1.0],1
1,m tired of,[0.9990883375593951],2
2,today ugh I,[0.9954454202941728],3
3,so tired its,[0.9927443986968594],4
4,thay I t,[0.9870877058809004],5
...,...,...,...
80773,get everything instal,[0.3003297649610356],80774
80774,chloroform or otherwise,[0.3002621834336448],80775
80775,class in my,[0.3001163944259027],80776
80776,small gaming comp,[0.30005560137779075],80777


In [67]:
# temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# # create a dataframe with n-gram, average score
# avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] + temp2['score'] + temp3['score'])/3)), columns=['keyphrase', 'avg_score'])
# avg_score_df

In [68]:
# avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
# avg_score_df

In [69]:
# Computing threshold

# avg_scores = avg_score_df['avg_score'].tolist()

# dk = 0
# for i in range(len(avg_scores)-1):
#     dk += abs(avg_scores[i+1] - avg_scores[i])

# dk = dk/(len(avg_scores)-1)

# sk = 0
# for i in range(len(avg_scores)-1):
#     sk += abs(abs(avg_scores[i+1] - avg_scores[i]) - dk)

# sk = sk/(len(avg_scores)-1)
# sk = np.sqrt(sk)

# threshold = sk/dk
# threshold = threshold[0]

# print("d(k) = ", dk)
# print("s(k) = ", sk)
# print("Threshold = ", threshold)

In [70]:
# threshold = 0.5

In [71]:
# anomalies = avg_score_df[avg_score_df['avg_score'] < threshold]['keyphrase'].tolist()
# anomalies

In [72]:
# remove these keyphrases from xlm_rank_df, distilbert_rank_df and minilm_rand_df
# xlm_rank_df = xlm_rank_df[~xlm_rank_df['keyphrase'].isin(anomalies)]
# distilbert_rank_df = distilbert_rank_df[~distilbert_rank_df['keyphrase'].isin(anomalies)]
# minilm_rank_df = minilm_rank_df[~minilm_rank_df['keyphrase'].isin(anomalies)]

## Processing to get the array to perform Rank Aggregation

In [73]:
# sort xlm_rank_df, distilbert_rank_df, minilm_rank_df by alphabetical order of n-grams
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# form a matrix where columns are the keyphrases and three rows, each row representing the rank of the keyphrase in each model
keyphrase_matrix = pd.DataFrame(columns=['xlm_rank', 'distilbert_rank', 'minilm_rank'])
keyphrase_matrix['xlm_rank'] = temp1['rank']
keyphrase_matrix['distilbert_rank'] = temp2['rank']
keyphrase_matrix['minilm_rank'] = temp3['rank']

m = keyphrase_matrix.to_numpy()
m = m.T
m

array([[ 3825, 18638, 25026, ..., 59547, 55080, 42975],
       [ 7410,  8711,  5076, ..., 10384, 15413, 13035],
       [ 8220,  8855, 23121, ..., 26757, 63452, 59440]])

In [74]:
m.shape

(3, 80778)

## Rank Aggregation using `ranky` to get the final rank for each keyphrase

In [75]:
# agg_ranks = rk.kemeny_young(m)
# agg_ranks

## Aggregate Rank
* Rank aggregation using `ranky` didn't work properly. So we just used average scores to give the final ranks.

In [76]:
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# create a dataframe with n-gram, average score
avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] * 0.45 + temp2['score'] * 0.3 + temp3['score'] * 0.25))), columns=['keyphrase', 'avg_score'])
avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
avg_score_df

,keyphrase,avg_score
0,sad I wolud,[0.8553206333984231]
1,I m suckere,[0.8328261630541444]
2,tonight but I,[0.8294227565626038]
3,yesterday sorry naaaattttyyy,[0.822091124621317]
4,hey I miss,[0.8168507324024123]
...,...,...
80773,the bar form,[0.3138091840974049]
80774,of sunflower seed,[0.311591656706377]
80775,point dollar be,[0.3107448487299758]
80776,email their website,[0.3076900271293228]


In [77]:
final_rank_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
final_rank_df['rank'] = final_rank_df.index + 1
final_rank_df

,keyphrase,avg_score,rank
0,sad I wolud,[0.8553206333984231],1
1,I m suckere,[0.8328261630541444],2
2,tonight but I,[0.8294227565626038],3
3,yesterday sorry naaaattttyyy,[0.822091124621317],4
4,hey I miss,[0.8168507324024123],5
...,...,...,...
80773,the bar form,[0.3138091840974049],80774
80774,of sunflower seed,[0.311591656706377],80775
80775,point dollar be,[0.3107448487299758],80776
80776,email their website,[0.3076900271293228],80777


In [78]:
final_rank_df.head(20)

,keyphrase,avg_score,rank
0,sad I wolud,[0.8553206333984231],1
1,I m suckere,[0.8328261630541444],2
2,tonight but I,[0.8294227565626038],3
3,yesterday sorry naaaattttyyy,[0.822091124621317],4
4,hey I miss,[0.8168507324024123],5
5,sorry I ll,[0.8117994217922222],6
6,I m terribly,[0.810974228128113],7
7,t sorry hopefully,[0.8091492126515726],8
8,it but I,[0.8089823866242122],9
9,boredd gah I,[0.8061210133706456],10


## Comparison with `KeyBERT`

In [79]:
concatenated_tweets = ' '.join(raw_tweets)
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(concatenated_tweets, keyphrase_ngram_range=(3, 3), top_n=100)
keywords

[('sick today sad', 0.4716),
 ('raining just bawled', 0.4663),
 ('hope feeling better', 0.4612),
 ('hope feel better', 0.4561),
 ('bummed night upsetting', 0.4553),
 ('bummed sick week', 0.4502),
 ('tonight missed sobs', 0.45),
 ('sprained glad im', 0.4485),
 ('upsetting ami agony', 0.4478),
 ('feeling better thursday', 0.4478),
 ('sooo sick today', 0.4409),
 ('feel better pally', 0.4403),
 ('raining today aww', 0.4387),
 ('definitely sore today', 0.4361),
 ('felt better thursday', 0.4353),
 ('feel better day', 0.435),
 ('sick excuse broke', 0.4342),
 ('today hurts really', 0.4332),
 ('feel better ll', 0.4321),
 ('sick today really', 0.4321),
 ('bummed today absolutely', 0.4318),
 ('sucks feeling weather', 0.4316),
 ('tonight mega bummed', 0.4315),
 ('feeling sorry today', 0.4314),
 ('lol sick today', 0.4301),
 ('hella sick today', 0.4296),
 ('upset missed chat', 0.4277),
 ('completely drained today', 0.427),
 ('feeling better stopped', 0.4252),
 ('missed sobs going', 0.4243),
 ('reall

## Metrics of the proposed model w.r.t KeyBERT outputs as ground truth

In [80]:
ground_truth = [x[0] for x in keywords]
ground_truth

['sick today sad',
 'raining just bawled',
 'hope feeling better',
 'hope feel better',
 'bummed night upsetting',
 'bummed sick week',
 'tonight missed sobs',
 'sprained glad im',
 'upsetting ami agony',
 'feeling better thursday',
 'sooo sick today',
 'feel better pally',
 'raining today aww',
 'definitely sore today',
 'felt better thursday',
 'feel better day',
 'sick excuse broke',
 'today hurts really',
 'feel better ll',
 'sick today really',
 'bummed today absolutely',
 'sucks feeling weather',
 'tonight mega bummed',
 'feeling sorry today',
 'lol sick today',
 'hella sick today',
 'upset missed chat',
 'completely drained today',
 'feeling better stopped',
 'missed sobs going',
 'really sick today',
 'today sprained thank',
 'wow freezing today',
 'read sigh rain',
 'feel better heeps',
 'help feel better',
 'rain today got',
 'rough today soooo',
 'hurts today uncomfortable',
 'feel better soon',
 'll feel better',
 'ua loss embarrassing',
 'hope tired today',
 'feeling bette

In [81]:
precision = 0

for keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        precision += 1

print("Proposed Model Precision: ", precision)

Proposed Model Precision:  0


In [82]:
recall = 0

for keyphrase in ground_truth:
    if keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
        recall += 1

print("Proposed Model Recall: ", recall)

Proposed Model Recall:  0


In [83]:
xlm_precision = 0
for keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        xlm_precision += 1

print("xlm-roberto Precision: ", xlm_precision)

distilbert_precision = 0
for keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        distilbert_precision += 1

print("Distilbert Precision: ", distilbert_precision)

minilm_precision = 0
for keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        minilm_precision += 1

print("Mini LM Precision: ", minilm_precision)

xlm-roberto Precision:  0
Distilbert Precision:  0
Mini LM Precision:  0


In [84]:
xlm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
        xlm_recall += 1

print("xlm-roberto Recall: ", xlm_recall)

distilbert_recall = 0
for keyphrase in ground_truth:
    if keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
        distilbert_recall += 1

print("distilbert Recall: ", distilbert_recall)

minilm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
        minilm_recall += 1

print("Mini LM Recall: ", minilm_recall)

xlm-roberto Recall:  0
distilbert Recall:  0
Mini LM Recall:  0


In [85]:
import scipy.stats

def rank_correlation(rank1, rank2):
  
    rho, pval = scipy.stats.spearmanr(rank1, rank2)
    return rho

# model_ranking = ["keyphrase1", "keyphrase2", "keyphrase3"]
# ground_truth = ["keyphrase2", "keyphrase1", "keyphrase4"]
similarity = rank_correlation(final_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of proposed model with KeyBERT: ", similarity)

spearman similarity of proposed model with KeyBERT:  -0.04009600960096008


In [86]:
xlm_similarity = rank_correlation(xlm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of xlm-roberto with KeyBERT: ", xlm_similarity)

distilbert_similarity = rank_correlation(distilbert_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of distilbert model with KeyBERT: ", distilbert_similarity)

minilm_similarity = rank_correlation(minilm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of Mini LM model with KeyBERT: ", minilm_similarity)

spearman similarity of xlm-roberto with KeyBERT:  0.02087008700870087
spearman similarity of distilbert model with KeyBERT:  -0.03636363636363636
spearman similarity of Mini LM model with KeyBERT:  0.06513051305130513
